# Web Scraping Sample Patient Unstructred Data

From the Agency for Healthcare Research and Quality.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import os


### Part I: Web Scraping Tool
Creating a function that web scrapes from multiple links (this is the unstructured data), parses the HTML (this is the ETL), and stores in a pandas DataFrame. The DataFrame gets convered to a CSV and a job scheduler imports it to the cloud based database. 

In [2]:
def web_scrape_page(url):
    resp = requests.get(url)
    
    # All info from webpage scraped
    soup = BeautifulSoup(resp.text,'html.parser')
    
    # Scrap all text in the data
    medrec_text = soup.find_all('p')
    
    # Scrap all text in tables in data
    table_values = []
    for line in soup.findAll('tr'):
        for l in line.findAll('td'):
            if l.find('sup'):
                l.find('sup').extract()
            table_values.append(l.getText())
            
    # Scrap Patient Name 
    name = medrec_text[5].get_text()
    
    # Scrap Patient Gender 
    gender = re.findall(r"\s(.*)", table_values[15])[0]
    
    # Scrap Patient D.O.B 
    dob = re.findall(r"\s(.*)", table_values[13])[0]
    dob = re.findall(r"\s(.*)", dob)[0]
    
    # Scrap Patient Race
    race = re.findall(r"\s(.*)", table_values[18])[0]
    
    # Scrap Patient SSN
    ssn = re.findall(r"\s(.*)", table_values[19])[0]
    ssn = re.findall(r"\s(.*)", ssn)[0]
    ssn = re.findall(r"\s(.*)", ssn)[0]
    
    # Scrap Patient Marital Status
    marital_status = re.findall(r"\s(.*)", table_values[16])[0]
    marital_status = re.findall(r"\s(.*)", marital_status)[0]
    
    # Scrap Date of Appt.
    date = table_values[30]
    
    # Scrap Patient Problems
    problems = medrec_text[7].get_text().replace("\n", ", ")
    
    # Scrap Patient Medication
    medication = medrec_text[8].get_text().replace("\n", ", ")
    
    # Scrap Patient Directives
    directives = medrec_text[9].get_text().replace("\n", ", ")
    
    # Scrap Patient Allergies
    allergies = medrec_text[11].get_text().replace("\n", ", ")
    
    # Scrap Patient Services Due
    services = medrec_text[13].get_text()
    
    # Scrap Patient Social history
    social_history = re.findall(r"\s(.*)", medrec_text[17].get_text())[0]
    social_history = re.findall(r"\s(.*)", social_history)[0]
    
    # Scrap Patient Vitals
    first_table = table_values[45:87]
    vitals = first_table[0::3]
    results = first_table[1::3]
    info = first_table[2::3]

    results2 = []
    for i in results:
        results2.append(i.replace("\xa0", ""))

    vitals2 = []
    for i in vitals:
        vitals2.append(i.replace("\xa0", ""))

    medrec_vitals = pd.DataFrame({'vitals': vitals2, 'vitals_results': results2, "vitals_info": info})
    
    # Scrap Patient Test Results
    last_table = table_values[97:]
    test = []
    results = []

    for i in range(len(last_table)):
        if i%2!= 0:
            results.append(last_table[i])
        else:
            test.append(last_table[i])
    medrec_tests = pd.DataFrame({'test': test, 'test_results': results})
        
    # Creating DataFrame
    df = pd.DataFrame([name,gender,dob,race,ssn,marital_status,date,problems,medication,directives,allergies,services,social_history]).transpose()
    df.columns = ['Name', 'Gender', 'Date of Birth', 'Race', 'SSN','Marital Status','Date Seen','Problems','Medication', 'Directives','Allergies','Services','Social History']
    
    medrec_vitals = medrec_vitals.T
    new_header = medrec_vitals.iloc[0] 
    medrec_vitals.columns = new_header 
    medrec_vitals = pd.DataFrame(medrec_vitals.iloc[1,:]).T
    medrec_vitals = medrec_vitals.rename(index={'vitals_results':0})
    
    medrec_tests = medrec_tests.T
    new_header = medrec_tests.iloc[0] 
    medrec_tests.columns = new_header 
    medrec_tests = pd.DataFrame(medrec_tests.iloc[1,:]).T
    medrec_tests = medrec_tests.rename(index={'test_results':0})
    
    final_df = pd.concat([df, medrec_vitals, medrec_tests], axis=1, sort=False)
    
    return final_df

In [3]:
pg1 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-adam-pie.html")
pg2 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-n-bill-windows.html")
pg3 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-billy-gato.html")
pg4 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-cherie-amore.html")
pg5 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-john-donut.html")
pg6 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-monica-latte.html")
pg7 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-steve-apple.html")
pg8 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-tom-gellato.html")
pg9 = web_scrape_page("https://www.ahrq.gov/ncepcr/tools/pf-handbook/mod8-app-b-wendy-see.html")


frames = [pg1, pg2, pg3, pg4, pg5, pg6, pg7, pg8, pg9]
med_recs = pd.concat(frames)
med_recs = med_recs.reset_index(drop=True)

In [4]:
# Save med_recs as csv
cwd = os.getcwd()
output_path = cwd + '\Med_Recs.csv'
med_recs.to_csv(output_path, index = True)